#### 1. Установить implicit;
#### 2. Взять датасет last.fm (урезанный или полный);
#### 3. Разбить датасет на обучающую и тестовую выборки;

In [93]:
import pandas as pd
import numpy as np

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    "data/rs/lastfm_small.tsv",
    sep="\t",
    header=None,
    names=col_names
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [94]:
data.fillna("None", inplace=True)
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [95]:
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [96]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 2
нет в тестовой выборке, но есть в обучающей: 24
всего пользователей: 20465


In [97]:
import warnings
warnings.filterwarnings('ignore')

In [98]:
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

#### 4. Построить на обучающей выборке хотя бы две модели из пакета implicit:

In [99]:
! rm -rf data/rs/tmp/libfm && mkdir -p data/rs/tmp/libfm && cd data/rs/tmp/libfm && mkdir train

In [100]:
data_dir = 'data/rs/tmp/libfm/'

In [101]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    data_dir + test_file_name,
    sep="\t",
    header=False,
    index=False
)
train_file_name = "train/lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    data_dir + train_file_name,
    sep="\t",
    header=False,
    index=False
)

In [102]:
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [103]:
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799831
Доля ненулевых элементов: 0.0005850443645104982
Среднее значение ненулевых элементов: 216.89491271972656
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


kNN по косинусной мере

In [104]:
! cd data/rs/tmp/libfm && mkdir -p recs1

In [105]:
%%time
from implicit.nearest_neighbours import CosineRecommender

model1 = CosineRecommender()
model1.fit(plays)
sparse_info(model1.similarity)

Размерности матрицы: (66800, 66800)
Ненулевых элементов в матрице: 1216906
Доля ненулевых элементов: 0.00027271191150632867
Среднее значение ненулевых элементов: 0.4430096204179387
Максимальное значение ненулевых элементов: 1.9999999999999996
Минимальное значение ненулевых элементов: 0.0
CPU times: user 652 ms, sys: 4 ms, total: 656 ms
Wall time: 268 ms


ALS

In [106]:
! cd data/rs/tmp/libfm && mkdir -p recs2

In [107]:
%%time
from implicit.als import AlternatingLeastSquares
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'
model2 = AlternatingLeastSquares(factors=50)
model2.fit(plays.astype(np.double))

CPU times: user 22.1 s, sys: 18.1 s, total: 40.2 s
Wall time: 11 s


#### 5. Получить рекомендации на тестовой выборке для обученных моделей;

In [108]:
%%time
print("получаем рекомендации для всех пользователей")
user_plays = plays.T.tocsr()
with open(data_dir + "recs1/" + test_file_name + ".recs.tsv", "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model1.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

получаем рекомендации для всех пользователей
CPU times: user 10.4 s, sys: 108 ms, total: 10.5 s
Wall time: 10.5 s


In [109]:
%%time
print("получаем рекомендации для всех пользователей")
user_plays = plays.T.tocsr()
with open(data_dir + "recs2/" + test_file_name + ".recs.tsv", "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model2.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))

получаем рекомендации для всех пользователей
CPU times: user 1min 35s, sys: 51.1 s, total: 2min 27s
Wall time: 38.4 s


#### 6. Сравнить метрики качества обученных моделей на тестовой выборке с помощью mrec (или иным способом).

In [110]:
! cd data/rs/tmp/libfm && mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train lastfm.test.0 \
    --recsdir recs1

[2017-12-10 23:09:00,354] INFO: processing /home/mitya/projects/homework/data/rs/tmp/libfm/lastfm.test.0...
None
mrr            0.0496 +/- 0.0000
prec@5         0.0168 +/- 0.0000
prec@10        0.0153 +/- 0.0000
prec@15        0.0102 +/- 0.0000
prec@20        0.0076 +/- 0.0000


In [111]:
! cd data/rs/tmp/libfm && mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train lastfm.test.0 \
    --recsdir recs2

[2017-12-10 23:09:03,513] INFO: processing /home/mitya/projects/homework/data/rs/tmp/libfm/lastfm.test.0...
None
mrr            0.3011 +/- 0.0000
prec@5         0.1313 +/- 0.0000
prec@10        0.1078 +/- 0.0000
prec@15        0.0719 +/- 0.0000
prec@20        0.0539 +/- 0.0000
